In [1]:
import pandas as pd
import json

In [2]:
f = open('../data/train.json')
data=json.load(f)
f = open('../data/test.json')
test=json.load(f)


In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts/')))

from prep_data import preprocess
from extract_entities import extract_entities


In [4]:
prep = preprocess(data)

In [5]:
def prepare_prompt(data,prompt_data):
    prompt = prompt_data['document']
    if len(data)==0:
        return 'extract entities from the following paragraphs?\n' + prompt + '\nExtracted Features:'
    if type(data) == list:
        p = preprocess(data)
    else: p = preprocess([data])
        
    
    examples=[]
    # data = [data]
    for i in range(len(data)):
        examples.append(p.get_final_template(i))
    final = 'extract entities from the following paragraphs?\n' + '\n--\n'.join(examples) + '\n--\n'+ prompt + '\nExtracted Features:'
    return final
    

In [6]:
def optimal_example(data,start,end):
    #compute the accuracy values for each size of data
    # let's select the last data as a test data
    query = data[-2]
    extract = extract_entities()
    prep = preprocess([query])
    result = prep.get_entities(0)
    r = prep.get_final_template(0).replace('description:'+result['document']+'\nExtracted Features:\n','')
    del(result['document'])
    
    accuracy = []
    examples = []
    print('label: ')
    print(r)
    for i in range(start,end):
        examples.append(i+1)
        prompt = prepare_prompt(data[:i+1],query)
        res = extract.predict(prompt)
        res_prep = extract.get_feature(res)
        acc = extract.evaluate(prediction=res_prep,result=result)
        accuracy.append(acc)
        print('-------------\nexample size:{} '.format(i+1))
        print(res)
    # return [examples,accuracy]
        

In [7]:
print(data[-2])


{'document': 'A successful member of our team will have: 5 + years of industry experience developing and implementing tools and applications at scale. BS/MS in Computer Science or equivalent industry experience. Demonstrated CS fundamentals with a deep appreciation for the differences between platforms and environments. Demonstrated technical design, problem solving and debugging skills. Unwavering passion to understand and simplify the developer experience. Fantastic communication skills. A track record of working collaboratively to understand customer needs and create solutions that positively impact our customers.', 'tokens': [{'text': '5 + years', 'start': 43, 'end': 52, 'token_start': 9, 'token_end': 11, 'entityLabel': 'EXPERIENCE'}, {'text': 'industry', 'start': 56, 'end': 64, 'token_start': 13, 'token_end': 13, 'entityLabel': 'SKILLS'}, {'text': 'developing and implementing tools', 'start': 76, 'end': 109, 'token_start': 15, 'token_end': 18, 'entityLabel': 'SKILLS'}, {'text': 'B

In [8]:
optimal_example(data,start=0,end=6)


label: 
DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5 + years in industry
SKILLS: industry, developing and implementing tools
-------------
example size:1 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years in developing
SKILLS: developing, tools, applications, scale
--
-------------
example size:2 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years in industry
SKILLS: industry experience, developing and implementing tools and applications at scale
-------------
example size:3 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years of industry experience
SKILLS: industry experience, CS fundamentals, technical design, problem
-------------
example size:4 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years in developing tools and applications at scale
SKILLS: developing tools and applications at scale
-------------
example size:5 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years of indu

In [9]:
optimal_example(data,start=6,end=12)

label: 
DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5 + years in industry
SKILLS: industry, developing and implementing tools
-------------
example size:7 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years in industry
SKILLS: industry, developing and implementing tools and applications, technical design
-------------
example size:8 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5 + years in developing and implementing tools and applications at scale
SKILLS: CS fundamentals
--


CohereError: too many tokens: total number of tokens (prompt and prediction) cannot exceed 2048 - received 2462. Try using a shorter prompt or a smaller max_tokens value.

### as we can see from the above error we can't increase the example size more than 8
- also example size 7 shows a good result so we will use that 

## Which Examples result good performance

In [21]:
def which_example(data,start,end):
    """ this method will evaluate which examples result in a good extraction performance"""
    query = data[-2]
    extract = extract_entities()
    prep = preprocess([query])
    result = prep.get_entities(0)
    r = prep.get_final_template(0).replace('description:'+result['document']+'\nExtracted Features:\n','')
    del(result['document'])
    
    accuracy = []
    examples = []
    print('label: ')
    print(r)
    for i in range(start,end):
        examples.append(i+1)
        prompt = prepare_prompt([data[i]],query)
        res = extract.predict(prompt)
        res_prep = extract.get_feature(res)
        acc = extract.evaluate(prediction=res_prep,result=result)
        accuracy.append(acc)
        print('-------------\nexample number:{} '.format(i))
        print(res)
    

In [22]:
which_example(data,1,len(data)-1)

label: 
DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5 + years in industry
SKILLS: industry, developing and implementing tools
-------------
example number:1 

DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years in industry
SKILLS: CS fundamentals, technical design, problem solving, debugging,
-------------
example number:2 

DIPLOMA: 
DIPLOMA_MAJOR: 
EXPERIENCE: 5+ years in industry
SKILLS: CS fundamentals
--
-------------
example number:3 

DIPLOMA: 
DIPLOMA_MAJOR: 
EXPERIENCE: 5+ years in industry experience developing and implementing tools and applications at scale
SKILLS: developing and implementing tools and
-------------
example number:4 

DIPLOMA: BS/MS in Computer Science or equivalent industry experience
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5 + years of industry experience developing and implementing tools and applications at scale
-------------
example number:5 

DIPLOMA: 
DIPLOMA_MAJOR: 
EXPERIENCE: 5+ years in industry
SKILLS: CS funda

In [ ]:
# as we can see from the above examples like 
# [ ]
# result a good performance so we will use 7 of such examples for the api

In [102]:
good_examples = [9,4,11,17,1,0,21]
sample_data = []
for i in good_examples:
    sample_data.append(data[i])

In [103]:
len(sample_data)

7

In [104]:
# Serializing json
# for i in sample_data:
#     json_object.append(json.dumps(i, indent=4))
 
json_object = json.dumps(sample_data,indent=4)

with open('../data/final_sample.json', "w") as text_file:
                text_file.write(json_object)
                # text_file.write('['+','.join(json_object)+']')